In [1]:
!pip install wget

import os
import json
import string
import wget
from zipfile import ZipFile

import numpy as np
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

!pip install sklearn_crfsuite
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer,confusion_matrix
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline

from pprint import pprint

from keras.preprocessing.text import Tokenizer

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9673 sha256=aa86ecc9a76c5f57fcc5c5f3caaf5ed8d3440498ba8fe714e139b1c280b4bba4
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
     |████████████████████████████████| 743 kB 7.2 MB/s 


In [2]:
# try:
#     from google.colab import files
# #   upload "train_PlayMusic_full.json" and "validate_PlayMusic.json" here
#     uploaded = files.upload()
#     train_loc = "train_PlayMusic_full.json"
#     test_loc = "validate_PlayMusic.json"
    
# except ModuleNotFoundError:
#     train_loc = "Data/snips/train_PlayMusic_full.json"
#     test_loc = "Data/snips/validate_PlayMusic.json"

!wget https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch6/Data/snips/train_PlayMusic_full.json
!wget https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch6/Data/snips/validate_PlayMusic.json

train_loc = "train_PlayMusic_full.json"
test_loc = "validate_PlayMusic.json"

train_file = json.load(open(train_loc, encoding= "iso-8859-2"))
test_file = json.load(open(test_loc, encoding= "iso-8859-2"))

--2021-08-07 15:17:48--  https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch6/Data/snips/train_PlayMusic_full.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 675343 (660K) [text/plain]
Saving to: ‘train_PlayMusic_full.json’

train_PlayMusic_ful 100%[===================>] 659.51K  --.-KB/s    in 0.03s   

2021-08-07 15:17:48 (23.2 MB/s) - ‘train_PlayMusic_full.json’ saved [675343/675343]

--2021-08-07 15:17:48--  https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch6/Data/snips/validate_PlayMusic.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:44

In [3]:
train_datafile = [i["data"] for i in train_file["PlayMusic"]]
test_datafile = [i["data"] for i in test_file["PlayMusic"]]

In [4]:
def convert_data(datalist):
    output = []
    count = 0
    for data in datalist:
      sent = []
      pos = []
      for phrase in data:
          words = phrase["text"].strip().split(" ")
          while "" in words:
              words.remove("")
          if "entity" in phrase.keys():
              label = phrase["entity"]
              labels = [label+"-{}".format(i+1) for i in range(len(words))]
          else:
              labels = ["O"] * len(words)
          sent.extend(words)
          pos.extend(labels)
      output.append([sent, pos])
      if count < 5:
        print(sent)
        count += 1
    return output

In [5]:
# '''convert_data test'''
# datalist = train_datafile
# output = []
# count = 0
# data = datalist[0]
# sent = []
# pos = []
# phrase = data[3]
# words = phrase["text"].strip().split(" ")
# while "" in words:
#     words.remove("")
# if "entity" in phrase.keys():
#     label = phrase["entity"]
#     labels = [label+"-{}".format(i+1) for i in range(len(words))]
# else:
#     labels = ["O"] * len(words)
# sent.extend(words)
# pos.extend(labels)

# print(words)
# print(labels)

In [6]:
train_data = convert_data(train_datafile)
test_data = convert_data(test_datafile)

['I', 'need', 'to', 'hear', 'the', 'song', 'Aspro', 'Mavro', 'from', 'Bill', 'Szymczyk', 'on', 'Youtube']
['play', 'Yo', 'Ho', 'from', 'the', 'new', 'york', 'pops', 'on', 'Youtube']
['Play', 'some', 'seventies', 'music', 'by', 'Janne', 'Puurtinen', 'on', 'Youtube', '.']
['play', 'the', 'MĂşsica', 'Da', 'SĂŠrie', 'De', 'Filmes', 'O', 'Hobbit', 'album', 'by', 'Alex', 'Otaola']
['Play', 'Magic', 'Sam', 'from', 'the', 'thirties']
['can', 'you', 'put', 'on', 'Like', 'A', 'Hurricane', 'by', 'Paul', 'Landers']
['Play', 'The', 'Happy', 'Blues', 'by', 'Ronnie', 'Wood', '.']
['Play', 'the', 'newest', 'melody', 'on', 'Last', 'Fm', 'by', 'Eddie', 'Vinson', '.']
['Use', 'Groove', 'Shark', 'to', 'play', 'music']
['Please', 'play', 'something', 'good', 'from', 'U-roy', '.', 'Any', 'song', 'from', '1975', 'on', 'Zvooq', 'will', 'do.']


In [7]:
try :
    from google.colab import files
    !wget -P glove.6B http://nlp.stanford.edu/data/glove.6B.zip
    !unzip glove.6B/glove.6B.zip -d glove.6B
    BASE_DIR='.'
except ModuleNotFoundError :
    if not os.path.exists(os.getcwd()+'\\Data\\glove.6B'):
            os.mkdir(os.getcwd()+'\\Data\\glove.6B')

            url='http://nlp.stanford.edu/data/glove.6B.zip' 
            path=os.getcwd()+'\Data' 
            wget.download(url,path)  

            temp=path+'\glove.6B.zip' 
            file = ZipFile(temp)  
            file.extractall(path+'\glove.6B') 
            file.close()   
   
    BASE_DIR = 'Data'  
    
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
MAX_SEQUENCE_LENGTH = 300
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.3

--2021-08-07 15:17:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-08-07 15:17:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-08-07 15:17:49--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B/glove.6B.zip’

In [8]:
'''look at first line'''
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf-8") as f:
    for line in f:
      print(line)
      break

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062



In [9]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))

def get_embeddings(word):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is None:
        # words not found in embedding index will be all-zeros.
        embedding_vector = np.zeros(shape=(EMBEDDING_DIM, ))
    return embedding_vector

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.


In [10]:
train_texts = [" ".join(i[0]) for i in train_data]
test_texts = [" ".join(i[0]) for i in test_data]

train_texts[0]

'I need to hear the song Aspro Mavro from Bill Szymczyk on Youtube'

In [11]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 2866 unique tokens.


In [12]:
"""
Get features for all words in the sentence
Features:
- word context: a window of 2 words on either side of the current word, and current word.
- POS context: a window of 2 POS tags on either side of the current word, and current tag. 
input: sentence as a list of tokens.
output: list of dictionaries. each dict represents features for that word.
"""
def sent2feats(sentence):
    feats = []
    sen_tags = pos_tag(sentence) #This format is specific to this POS tagger!
    for i in range(0,len(sentence)):
        word = sentence[i]
        wordfeats = {}
       #word features: word, prev 2 words, next 2 words in the sentence.
        wordfeats['word'] = word
        if i == 0:
            wordfeats["prevWord"] = wordfeats["prevSecondWord"] = "<S>"
        elif i==1:
            wordfeats["prevWord"] = sentence[0]
            wordfeats["prevSecondWord"] = "</S>"
        else:
            wordfeats["prevWord"] = sentence[i-1]
            wordfeats["prevSecondWord"] = sentence[i-2]
        #next two words as features
        if i == len(sentence)-2:
            wordfeats["nextWord"] = sentence[i+1]
            wordfeats["nextNextWord"] = "</S>"
        elif i==len(sentence)-1:
            wordfeats["nextWord"] = "</S>"
            wordfeats["nextNextWord"] = "</S>"
        else:
            wordfeats["nextWord"] = sentence[i+1]
            wordfeats["nextNextWord"] = sentence[i+2]
        
        #POS tag features: current tag, previous and next 2 tags.
        wordfeats['tag'] = sen_tags[i][1]
        if i == 0:
            wordfeats["prevTag"] = wordfeats["prevSecondTag"] = "<S>"
        elif i == 1:
            wordfeats["prevTag"] = sen_tags[0][1]
            wordfeats["prevSecondTag"] = "</S>"
        else:
            wordfeats["prevTag"] = sen_tags[i - 1][1]

            wordfeats["prevSecondTag"] = sen_tags[i - 2][1]
            # next two words as features
        if i == len(sentence) - 2:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = "</S>"
        elif i == len(sentence) - 1:
            wordfeats["nextTag"] = "</S>"
            wordfeats["nextNextTag"] = "</S>"
        else:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = sen_tags[i + 2][1]
            
        #Adding word vectors
        vector = get_embeddings(word)
        for iv,value in enumerate(vector):
            wordfeats['v{}'.format(iv)]=value
        
        
        feats.append(wordfeats)
    return feats

In [13]:
#Extract features from the conll data, after loading it.
def get_feats_conll(conll_data):
    feats = []
    labels = []
    for sentence in conll_data:
        feats.append(sent2feats(sentence[0]))
        labels.append(sentence[1])
    return feats, labels

In [14]:
#Train a sequence model
def train_seq(X_train,Y_train,X_dev,Y_dev):
    crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)#, all_possible_states=True)
    #Just to fit on training data
    crf.fit(X_train, Y_train)
    labels = list(crf.classes_)
    #testing:
    y_pred = crf.predict(X_dev)
    sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
    print(metrics.flat_f1_score(Y_dev, y_pred,average='weighted', labels=labels))
    print(metrics.flat_classification_report(Y_dev, y_pred, labels=sorted_labels, digits=3))
    #print(metrics.sequence_accuracy_score(Y_dev, y_pred))
    get_confusion_matrix(Y_dev, y_pred,labels=sorted_labels)


In [15]:
#source for this function: https://gist.github.com/zachguo/10296432
def print_cm(cm, labels):
    print("\n")
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        sum = 0
        for j in range(len(labels)):
            cell = "%{0}.0f".format(columnwidth) % cm[i, j]
            sum =  sum + int(cell)
            print(cell, end=" ")
        print(sum) #Prints the total number of instances per cat at the end.


In [16]:
#python-crfsuite does not have a confusion matrix function, 
#so writing it using sklearn's confusion matrix and print_cm from github
def get_confusion_matrix(y_true,y_pred,labels):
    trues,preds = [], []
    for yseq_true, yseq_pred in zip(y_true, y_pred):
        trues.extend(yseq_true)
        preds.extend(yseq_pred)
    print_cm(confusion_matrix(trues,preds,labels),labels)


In [41]:
# '''test lambda sorter'''
# sorter = lambda name: (name[1:], name[0])

# for name in labels:
#   print(sorter(name))

In [47]:
# '''test train_seq function'''
# print(train_data[0][0])
# print(feats[0])

# X_train,Y_train,X_dev,Y_dev = feats, labels, devfeats, devlabels
# crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)#, all_possible_states=True)
# #Just to fit on training data
# crf.fit(X_train, Y_train)
# labels = list(crf.classes_)
# #testing:
# y_pred = crf.predict(X_dev)
# sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
# print(metrics.flat_f1_score(Y_dev, y_pred,average='weighted', labels=labels))
# print(metrics.flat_classification_report(Y_dev, y_pred, labels=sorted_labels, digits=3))
# print(metrics.sequence_accuracy_score(Y_dev, y_pred))
# get_confusion_matrix(Y_dev, y_pred,labels=sorted_labels)

In [17]:
print("Training a Sequence classification model with CRF")
feats, labels = get_feats_conll(train_data)
devfeats, devlabels = get_feats_conll(test_data)
train_seq(feats, labels, devfeats, devlabels)
print("Done with sequence model")

Training a Sequence classification model with CRF


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8560889758746073
              precision    recall  f1-score   support

           O      0.976     0.983     0.980       418
      year-1      0.960     0.960     0.960        25
     genre-1      1.000     0.333     0.500         3
     genre-2      1.000     0.333     0.500         3
     genre-3      0.000     0.000     0.000         0
     genre-4      0.000     0.000     0.000         0
     genre-5      0.000     0.000     0.000         0
     genre-6      0.000     0.000     0.000         0
   service-1      0.857     0.923     0.889        39
   service-2      1.000     0.818     0.900        11
  playlist-1      1.000     0.333     0.500         9
  playlist-2      0.333     0.167     0.222         6
  playlist-3      0.000     0.000     0.000         4
  playlist-4      0.000     0.000     0.000         4
  playlist-5      0.000     0.000     0.000         2
  playlist-6      0.000     0.000     0.000         0
     album-1      0.000     0.000     0.000        13
     alb